In [1]:
'''
This package is needed to run the code
'''

!pip install scikit-image

In [2]:
'''
We import the libraries and define the functions 
that will be required for calculating the FID
We make proper citation to this post: https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/
which we made use of
'''
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10
 
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    
    # we added this line to help get a sense of model output
    print("SHAPE OF MODEL OUTPUT->", act1.shape)
    
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
 

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [3]:
'''
Having defined the models and preprocessing functions, 
we calculate FID on a pair of image collections
'''
import numpy as np
import glob
from PIL import Image

# pass the location to the first collection of images
filelist1 = glob.glob('/home/student/Music/DO_NOT_DELETE_Shinkai_animeganplus/samples/063/*b.png')
image_list1 = [np.array(Image.open(fname)) for fname in filelist1]
images_list1_resized = list()
for image in image_list1:
    # resize with nearest neighbor interpolation
    new_image = resize(image, (299,299,3), 0)
    # store
    images_list1_resized.append(new_image)
images_list1_resized = np.array(images_list1_resized)
print(images_list1_resized.shape)  

# pass the location to the second collection of images
filelist2 = glob.glob('/home/student/Music/DO_NOT_DELETE_Paprika_animeganplus/AnimeGAN/AnimeGAN/dataset/Shinkai/style/*.jpg')
image_list2 = [np.array(Image.open(fname)) for fname in filelist2]
image_list2 = image_list2[:2000]
images_list2_resized = list()
for image in image_list2:
    # resize with nearest neighbor interpolation
    new_image = resize(image, (299,299,3), 0)
    # store
    images_list2_resized.append(new_image)
images_list2_resized = np.array(images_list2_resized)
print(images_list2_resized.shape)   



# pre-process images
images_list1_resized = preprocess_input(images_list1_resized)
images_list2_resized = preprocess_input(images_list2_resized)

# calculate fid from both collections of images inception model
fid = calculate_fid(model, images_list1_resized, images_list2_resized)
print('FID: %.3f' % fid)


(67, 299, 299, 3)
(1650, 299, 299, 3)

SHAPE OF MODEL OUTPUT-> (67, 2048)
FID: 2.083
